In [22]:
import time
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import pandas as pd
import numpy as np

from keras.utils import image_dataset_from_directory
from tensorflow import keras 
from tensorflow.keras.utils import to_categorical
from keras import layers
from keras import models
from keras.callbacks import EarlyStopping
from tensorflow.data import Options


import psutil
import subprocess
import platform

### Configuração da GPU no TensorFlow

Antes de iniciar o treino do modelo, é importante garantir que o *TensorFlow* está configurado para utilizar a GPU (caso esteja disponível). Além disso, ativamos o *memory growth*, que permite ao *TensorFlow* alocar memória da GPU conforme necessário, evitando reservar toda a memória de uma vez.

In [ ]:
# Verificar dispositivos físicos do tipo 'GPU' disponíveis
gpus = tf.config.list_physical_devices('GPU')

# Se houver GPUs disponíveis, configurar o memory growth
if gpus:
    try:
        for gpu in gpus:
            # Ativar crescimento dinâmico da memória da GPU
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled.")
    except RuntimeError as e:
        # Caso a GPU já tenha sido inicializada, não é possível alterar a configuração
        print(e)

### Exploração da Estrutura de Diretorias

Antes de carregar os dados, é importante garantir que o caminho para os ficheiros está correto e que a estrutura de diretorias está bem organizada. O seguinte bloco de código permite:

- Definir o caminho base (`root_path`) para o projeto.
- Listar as diretorias existentes na raiz do projeto.
- Verificar se a diretoria do dataset (`garbage-noaug-70-15-15`) existe e visualizar o seu conteúdo.
- Explorar de forma recursiva a estrutura de diretorias, mostrando ficheiros e pastas com indentação hierárquica.

Este passo é essencial para:
- Validar que os dados estão organizados corretamente.
- Evitar erros de caminho ao carregar imagens para treino, validação e teste.

In [ ]:
# Caminho local para a pasta raiz do projeto
root_path = "./"  

# Listar diretorias no caminho raiz
print("Diretorias no caminho raiz:")
print(os.listdir(root_path))

# Verificar conteúdo de um caminho específico
specific_path = os.path.join(root_path, "garbage-noaug-70-15-15")
if os.path.exists(specific_path):
    print(f"\n Conteúdo de {specific_path}:")
    print(os.listdir(specific_path))
else:
    print(f"\n Caminho {specific_path} não existe")

# Função para listar diretorias com profundidade
def list_dirs(path, indent=0):
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        if os.path.isdir(full_path):
            print(" " * indent + "📁 " + item)
            if indent < 4:
                list_dirs(full_path, indent + 2)
        else:
            print(" " * indent + "📄 " + item)

# Explorar estrutura de diretorias
print("\n Estrutura de diretorias:")
list_dirs(root_path, 0)

### Deteção e Configuração Otimizada de GPU (Apple Silicon / Metal)

Este bloco de código trata da deteção e configuração de dispositivos de aceleração como GPUs ou MPS (*Metal Performance Shaders*), especialmente útil em Macs com Apple Silicon.

#### Funcionalidades:
- Procura dispositivos GPU disponíveis (*TensorFlow* ≥ 2.5 reconhece *Metal* como `GPU`).
- Se não encontrar GPU, tenta encontrar dispositivos `MPS` diretamente.
- Ativa `memory growth` para evitar alocação antecipada excessiva de memória.
- Verifica e imprime os dispositivos visíveis.
- Executa uma multiplicação de matrizes simples para testar a aceleração via GPU.

In [ ]:
# Improved Metal GPU detection for Apple Silicon
try:
    # First try looking for GPU devices (newer TF versions label Metal as GPU)
    gpus = tf.config.list_physical_devices('GPU')
    if len(gpus) > 0:
        print(f"Found {len(gpus)} GPU device(s)")
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU acceleration enabled (Metal)")
    # If no GPU found, try looking specifically for MPS devices
    elif hasattr(tf.config, 'list_physical_devices') and len(tf.config.list_physical_devices('MPS')) > 0:
        mps_devices = tf.config.list_physical_devices('MPS')
        tf.config.experimental.set_visible_devices(mps_devices[0], 'MPS')
        print("MPS (Metal) device enabled")
    else:
        print("No GPU or MPS device found, using CPU")
        
    # Verify what device is being used
    print("\nDevice being used:", tf.config.get_visible_devices())
    
    # Test with a simple operation to confirm GPU usage
    with tf.device('/GPU:0'):
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
        c = tf.matmul(a, b)
        print("Matrix multiplication result:", c)
        print("GPU test successful!")
except Exception as e:
    print(f"Error setting up GPU: {e}")
    print("Falling back to CPU")

### Mixed Precision Training (FP16)

Este bloco de código ativa o **mixed precision training**, que usa `float16` (FP16) em vez de `float32` (FP32), sempre que possível.

#### Benefícios:
- Maior desempenho em GPUs modernas, como as da arquitetura Volta, Turing, Ampere ou Apple Silicon com suporte a *Metal*.
- Menor uso de memória, permitindo treinar modelos maiores.

#### Como funciona:
- Operações matemáticas intensas usam `float16`
- A perda (`loss`) e os pesos principais mantêm-se em `float32` para estabilidade numérica

In [26]:
# Enable mixed precision (faster on GPU)
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')  # Use FP16 instead of FP32


### Carregamento e Preparação dos Dados

Este bloco de código realiza a preparação do dataset antes do treino do modelo, incluindo carregamento das imagens, redimensionamento, *batching* e otimizações de desempenho.

#### Definição de Caminhos

```python
train_dir = specific_path + "/train"
validation_dir = specific_path + "/valid"
test_dir = specific_path + "/test"
```

Define os caminhos para os diretórios contendo os dados de treino, validação e teste. Espera-se que o `specific_path` aponte para a pasta raiz onde os dados estão organizados em subpastas por classe.

#### Configurações de Imagem

```python
IMG_SIZE = 128
BATCH_SIZE = 64
```

- `IMG_SIZE`: Redimensiona todas as imagens para 128x128. Apesar das imagens originais serem 640x640, reduzir o tamanho melhora a velocidade de treino e reduz o uso de memória.
- `BATCH_SIZE`: Define o número de imagens por batch. Um valor de 64 é eficiente para GPUs com memória moderada.

#### Carregamento do Dataset

Carrega as imagens a partir dos diretórios com as seguintes opções:

- Redimensionamento para o tamanho especificado.
- Organização automática dos dados por classes (com base nas subpastas).
- Conversão em batches para serem usados no treino.

São criados três datasets:

- `train_dataset`: para treino do modelo.
- `validation_dataset`: para avaliação durante o treino.
- `test_dataset`: para avaliação final.

#### *Prefetching* para Aceleração

```python
.prefetch(buffer_size=tf.data.AUTOTUNE)
```

Usa `prefetch` para carregar os dados no background enquanto o modelo treina, o que reduz o tempo de espera entre *batches* e melhora o desempenho.


#### Limitar Dados para Testes Rápidos

```python
train_dataset_pref = train_dataset_pref.take(100)
```

Limita o número de *batches* usados no treino. Útil durante testes rápidos para evitar longos tempos de treino enquanto se afina a arquitetura ou outros detalhes.

#### Otimizações Adicionais

```python
options = Options()
options.experimental_optimization.parallel_batch = False
```

Desativa o batching paralelo automático (por vezes usado por TensorFlow), o que pode ser necessário em ambientes com recursos limitados ou para evitar conflitos de performance.


In [ ]:
# Definição das diretorias de treino, validação e teste
train_dir = specific_path + "/train"
validation_dir = specific_path + "/valid"
test_dir = specific_path + "/test"

# Definir o tamanho das imagens e o tamanho do batch
# (Imagens originais têm 640px, mas 128px acelera o treino)
IMG_SIZE = 128
BATCH_SIZE = 64


# Carregar o dataset de treino a partir da diretoria
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE), # Redimensionar imagens
    batch_size=BATCH_SIZE            # Dividir em batches
)

# Carregar o dataset de validação
validation_dataset = image_dataset_from_directory(
    validation_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

# Carregar o dataset de teste
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

# Aplicar prefetching para otimizar o carregamento dos dados durante o treino
train_dataset_pref = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset_pref = validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset_pref = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Limitar o número de batches de treino para testes mais rápidos
train_dataset_pref = train_dataset_pref.take(100)

# Ajustar opções do dataset para evitar paralelismo automático no batching
options = Options()
options.experimental_optimization.parallel_batch = False
train_dataset_pref = train_dataset_pref.with_options(options)


## Verificação do Balanceamento das Classes

Antes de treinar o modelo, é importante compreender a distribuição de amostras entre as diferentes classes. Um dataset desbalanceado pode afetar negativamente a performance do modelo, especialmente em classes minoritárias.

O código abaixo calcula o número de exemplos por classe no dataset de treino (`train_dataset`), construindo um dicionário com contagens e depois convertendo essa informação num DataFrame para visualização.


In [ ]:
# Check class balance
class_counts = {}
# Itera sobre os batches do dataset de treino
for _, labels in train_dataset:
    for label in labels.numpy():
        if label not in class_counts:
            class_counts[label] = 0
        class_counts[label] += 1

# Cria um DataFrame com a contagem por classe
df = pd.DataFrame({'class': train_dataset.class_names, 'count': [class_counts.get(i, 0) for i in range(len(train_dataset.class_names))]})

# Mostra o DataFrame
print(df)


## Definição da Arquitetura CNN com Normalização por Batch e Otimizador RMSprop

Nesta secção, é apresentada a construção manual de uma rede neuronal convolucional (CNN) simples, composta por três blocos convolucionais seguidos de uma cabeça densa totalmente conectada. Esta arquitetura foi projetada com o objetivo de classificar imagens em múltiplas categorias, com base no dataset previamente carregado.

O pré-processamento inicial das imagens é feito com a camada `Rescaling`, que normaliza os valores de pixel para o intervalo `[0, 1]`. Esta transformação é essencial para acelerar a convergência do modelo e garantir estabilidade numérica durante o treino.

### Estrutura dos Blocos Convolucionais

Cada bloco convolucional é constituído por:

- Uma camada `Conv2D` com `padding='same'`, que mantém as dimensões espaciais da imagem,
- Uma camada `BatchNormalization`, que normaliza as ativações antes da função de ativação, estabilizando e acelerando o treino,
- Uma função de ativação `ReLU` para introdução de não-linearidade,
- Uma camada `MaxPooling2D`, responsável por reduzir a dimensão espacial e extrair as características mais salientes da imagem.

São empilhados três destes blocos, com 32, 64 e 128 filtros, respetivamente, aumentando progressivamente a capacidade de extração de padrões visuais.

### Camadas Densas e Classificação

Após a extração de características, os tensores são achatados (`Flatten`) e processados por uma camada densa (`Dense`) com 256 unidades, seguida de:

- Uma camada `BatchNormalization`,
- Uma função de ativação `ReLU`,
- Uma camada `Dropout` com taxa de 30%, para reduzir o risco de *overfitting*.

A camada de saída é uma `Dense` com ativação `softmax`, que gera uma distribuição de probabilidades sobre as classes disponíveis, permitindo assim a decisão final do modelo.

### Compilação do Modelo

O modelo foi compilado com:

- O otimizador `RMSprop`, utilizando uma taxa de aprendizagem de `1e-3`, adequada para tarefas de classificação com redes convolucionais,
- A função de perda `categorical_crossentropy`, apropriada para rótulos codificados em one-hot,
- A métrica de desempenho `accuracy`, usada para monitorizar a taxa de classificações corretas durante o treino e a validação.

### Resumo da Arquitetura

A arquitetura final do modelo foi visualizada através do método `model.summary()`, que apresenta o número total de parâmetros treináveis e a estrutura sequencial das camadas.


In [ ]:
# Definir a arquitetura da CNN com BatchNormalization antes da ativação
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = layers.Rescaling(1./255)(inputs)

# Primeira camada convolucional
x = layers.Conv2D(32, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D()(x)

# Segunda camada convolucional
x = layers.Conv2D(64, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D()(x)

# Terceira camada convolucional
x = layers.Conv2D(128, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D()(x)

# Camadas densas
x = layers.Flatten()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

# Camada de saída
outputs = layers.Dense(len(train_dataset.class_names), activation='softmax')(x)

# Criar o modelo
model = keras.Model(inputs=inputs, outputs=outputs)

# Otimizador
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3)

# Compilar o modelo
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumo do modelo
model.summary()

In [30]:
# timeout callback to stop training after a certain time limit
class TimeoutCallback(tf.keras.callbacks.Callback):
    def __init__(self, max_time_mins=2):
        super().__init__()
        self.max_time_sec = max_time_mins * 60
        self.start_time = time.time()
        
    def on_epoch_end(self, epoch, logs=None):
        elapsed = time.time() - self.start_time
        if elapsed > self.max_time_sec:
            print(f"\nReached time limit ({self.max_time_sec/3600:.1f}h). Stopping training.")
            self.model.stop_training = True

# Maximum 20 minutes of training to prevent overheating
timeout_cb = TimeoutCallback(max_time_mins=20)


In [31]:
# Callback para early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    patience=5,
    restore_best_weights=True
)

# Callback para salvar o modelo com melhor desempenho
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "models/model_checkpoint.keras", 
    save_best_only=True,
    monitor="val_accuracy"
)

# Add more callbacks for better training
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Calculate class weights
total = sum(class_counts.values())
class_weight = {i: total/count for i, count in class_counts.items()}


### Controlo de Tempo no Treino com *Callback* Personalizado

Durante o treino de modelos de *deep learning*, é comum que sessões de treino longas causem sobreaquecimento de dispositivos locais, especialmente em ambientes com recursos limitados, como computadores portáteis ou dispositivos integrados. Para mitigar este problema, foi implementado um **callback personalizado** que permite interromper o treino automaticamente após um determinado tempo máximo.

#### Descrição da Lógica

Foi criada uma classe chamada `TimeoutCallback`, que herda de `tf.keras.callbacks.Callback`. Esta classe é inicializada com um parâmetro `max_time_mins`, que define o tempo máximo de treino em minutos. O tempo de início é registado no momento da criação do *callback*.

A lógica de controlo é aplicada ao final de cada época (`on_epoch_end`). Nesta fase, é calculado o tempo decorrido desde o início do treino. Se o tempo exceder o limite especificado, o treino é interrompido automaticamente com a instrução `self.model.stop_training = True`.

Esta abordagem é particularmente útil em:
- Situações em que o treino é feito localmente e se pretende evitar uso excessivo do processador/GPU.
- Processos experimentais em que se deseja limitar o tempo de execução.
- Ambientes partilhados com restrições de tempo ou energia.

#### Parâmetros Utilizados

- `max_time_mins=20`: define um tempo máximo de treino de 20 minutos.
- O tempo é convertido para segundos (`max_time_sec`) para facilitar a comparação com o tempo decorrido.
- Ao atingir esse tempo, o treino termina automaticamente com uma mensagem de aviso no terminal.

In [32]:
class ResourceMonitorCallback(tf.keras.callbacks.Callback):
    def __init__(self, check_interval=1):
        super().__init__()
        self.check_interval = check_interval
        self.epoch_count = 0
        self.is_mac = platform.system() == 'Darwin'
        
    def on_epoch_end(self, epoch, logs=None):
        self.epoch_count += 1
        if self.epoch_count % self.check_interval == 0:
            # Get basic info
            cpu_percent = psutil.cpu_percent(interval=0.5)
            memory = psutil.virtual_memory()
            mem_used = f"{memory.percent}% ({memory.used / 1024**3:.1f}GB)"
            
            # Temperature check - simplified
            temp = "N/A"
            if self.is_mac:
                try:
                    # Try thermal level from pmset (no sudo needed)
                    result = subprocess.run(['pmset', '-g', 'therm'], capture_output=True, text=True)
                    if "CPU_Thermal_level" in result.stdout:
                        temp = result.stdout.strip()
                except: pass
            
            # Simplified output
            print(f"\n[Epoch {epoch}] CPU: {cpu_percent}% | Memory: {mem_used}")
            print(f"Thermal: {temp}")
            print(f"GPU: {'Active' if self.is_mac else 'Unknown'}")

# Create monitor that checks every epoch
resource_monitor = ResourceMonitorCallback(check_interval=1)


## Conversão dos Rótulos para One-Hot Encoding com `tf.data`

Para utilizar a função de perda `categorical_crossentropy`, é necessário que os rótulos estejam no formato **one-hot encoded**, ou seja, vetores binários com um `1` na posição da classe correta e `0` nas restantes. Este bloco de código demonstra como realizar essa transformação diretamente no pipeline `tf.data`.

### Obtenção do Número de Classes

```python
NUM_CLASSES = len(train_dataset.class_names)
```

É obtido dinamicamente o número total de classes com base nas subpastas do dataset, assumindo que `train_dataset` foi carregado com `image_dataset_from_directory(...)`.

### Definição da Função de Conversão

```python
def one_hot_encode(image, label):
    return image, tf.one_hot(label, depth=NUM_CLASSES)
```

A função `one_hot_encode` recebe a imagem e o rótulo inteiro, e converte o rótulo para um vetor one-hot com dimensão `NUM_CLASSES`.

### Aplicação ao Dataset

```python
train_dataset_oh = train_dataset_pref.map(one_hot_encode)
validation_dataset_oh = validation_dataset_pref.map(one_hot_encode)
```

A função de mapeamento é aplicada diretamente ao dataset com `.map(...)`, garantindo que todas as batches de treino e validação passem a conter rótulos compatíveis com a função de perda `categorical_crossentropy`.

Este método é eficiente e aproveita o paralelismo interno do TensorFlow, mantendo o dataset na forma de `tf.data.Dataset`, ideal para treino em GPU e integração com `model.fit(...)`.



In [34]:
# Número de classes (pode ser len(class_names))
NUM_CLASSES = len(train_dataset.class_names)

# Função para converter os rótulos para one-hot
def one_hot_encode(image, label):
    return image, tf.one_hot(label, depth=NUM_CLASSES)

# Aplicar ao dataset
train_dataset_oh = train_dataset_pref.map(one_hot_encode)
validation_dataset_oh = validation_dataset_pref.map(one_hot_encode)



## Estratégias de Regularização e Otimização durante o Treino

Durante o processo de treino do modelo, foram aplicadas diversas estratégias para melhorar a performance e evitar *overfitting*:

### *EarlyStopping*

Foi utilizado o *callback* `EarlyStopping` com os seguintes parâmetros:

- **monitor**: `val_accuracy` – o treino é monitorizado com base na precisão da validação.
- **patience**: `5` – se a métrica monitorizada não melhorar durante 5 épocas consecutivas, o treino é interrompido.
- **restore_best_weights**: `True` – garante que os pesos do modelo com melhor desempenho na validação sejam restaurados após o término do treino.

Esta técnica é útil para evitar *overfitting* e desperdício de recursos computacionais em épocas desnecessárias.

### *ModelCheckpoint*

Foi utilizado o *callback* `ModelCheckpoint` para guardar automaticamente o modelo com melhor desempenho de validação:

- **filepath**: `"models/model_checkpoint.keras"` – local onde o modelo é guardado.
- **save_best_only**: `True` – guarda apenas o modelo com melhor desempenho.
- **monitor**: `val_accuracy` – a métrica utilizada para definir o melhor modelo.

Este método assegura que, mesmo que o treino continue após o ponto ótimo, o melhor modelo está sempre disponível.

### *ReduceLROnPlateau*

Para ajustar dinamicamente a taxa de aprendizagem, foi aplicado o *callback* `ReduceLROnPlateau`:

- **monitor**: `val_loss` – reduz a `learning rate` com base na perda de validação.
- **factor**: `0.2` – a `learning rate` é multiplicada por este fator.
- **patience**: `3` – se não houver melhoria na perda durante 3 épocas, a taxa de aprendizagem é reduzida.
- **min_lr**: `1e-6` – taxa de aprendizagem mínima permitida.

Esta técnica melhora a convergência do modelo, especialmente em fases em que a otimização desacelera.

### Cálculo dos Pesos das Classes

Para mitigar o desbalanceamento entre classes no *dataset*, foram calculados pesos inversamente proporcionais à frequência de cada classe. O objetivo é forçar o modelo a prestar mais atenção às classes menos representadas, melhorando a generalização e o equilíbrio na predição.

O dicionário `class_weight` associa cada classe ao seu respetivo peso, baseado na fórmula:

```
peso_da_classe_i = total_de_amostras / número_de_amostras_na_classe_i
```

Este dicionário pode ser passado diretamente ao método `model.fit()` durante o treino.


In [ ]:
# train the model with early stopping and resource monitoring
history = model.fit(
    train_dataset_oh,
    validation_data=validation_dataset_oh,
    epochs=20,
    class_weight=class_weight,
    callbacks=[early_stopping, reduce_lr, checkpoint_cb, timeout_cb, resource_monitor]
)


# Save the entire model (architecture + weights + optimizer state)
model.save("models/garbage_classifier_model_early_stopping_v2.keras")  
model.save_weights("models/garbage_classifier_early_stopping_v2.weights.h5")
print("Model saved successfully!")

## Monitorização de Recursos durante o Treino

Com o objetivo de acompanhar o impacto computacional durante o processo de treino, foi implementada uma *callback* personalizada chamada `ResourceMonitorCallback`. Esta classe estende a API de `tf.keras.callbacks.Callback` e permite monitorizar periodicamente o uso de CPU, memória e temperatura do sistema, especialmente útil para ambientes de treino locais, como portáteis ou *workstations*.

### Funcionamento

A callback é ativada no final de cada época (`on_epoch_end`) e realiza as seguintes tarefas:

- **Uso de CPU**: Recolhe a percentagem de utilização da CPU utilizando a biblioteca `psutil`.
- **Uso de Memória**: Calcula a percentagem e a quantidade de memória RAM em uso (em GB).
- **Temperatura (macOS)**: Se o sistema for um Mac, tenta obter informações térmicas simplificadas através do comando `pmset` (sem necessidade de permissões elevadas).
- **Estado da GPU**: Mostra se há atividade reconhecida na GPU, sendo uma estimativa simplificada para sistemas macOS.

### Parâmetro `check_interval`

O construtor da classe permite configurar o parâmetro `check_interval`, que define a frequência (em número de épocas) com que a monitorização é realizada. Neste projeto, foi definido para verificar **a cada época** (`check_interval=1`), garantindo um controlo contínuo e detalhado ao longo do treino.

### Benefícios

Esta abordagem permite:
- Detetar sobrecargas de CPU ou memória que possam afetar a estabilidade do treino;
- Avaliar a eficiência dos recursos utilizados;
- Observar o comportamento térmico em dispositivos móveis (útil em laptops).

In [ ]:
# Plot training history with data from the callbacks
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(accuracy) + 1) # Adjusted to match the number of epochs

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, accuracy, 'bo-', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'ro-', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Training loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


### Treino do Modelo com Callbacks e Salvamento

Nesta etapa, o modelo é treinado utilizando diversas estratégias de regularização e monitorização para garantir um desempenho robusto e prevenir *overfitting* ou consumo excessivo de recursos.

#### Treinamento com `model.fit()`

O método `model.fit()` é invocado com os seguintes parâmetros:

- `train_dataset_pref`: conjunto de dados de treino, com prefetching e otimizações aplicadas.
- `validation_dataset_pref`: conjunto de validação para monitorizar o desempenho do modelo.
- `epochs=20`: o treino será realizado por um máximo de 20 épocas.
- `class_weight=class_weight`: pesos são aplicados às classes para corrigir desequilíbrios no número de exemplos por classe.
- `callbacks`: lista de funções auxiliares que atuam durante o treino, incluindo:
  - `early_stopping`: para interromper o treino quando a métrica de validação parar de melhorar;
  - `reduce_lr`: para diminuir a taxa de aprendizagem se o desempenho estagnar;
  - `checkpoint_cb`: para salvar o modelo sempre que atinge uma nova melhor performance;
  - `timeout_cb`: para limitar a duração do treino;
  - `resource_monitor`: para monitorar a memória e uso de GPU durante o treino.

#### Salvamento do Modelo

Após o treino:

- O modelo completo (arquitetura, pesos e estado do otimizador) é salvo no formato `.keras`, facilitando o reuso ou exportação.
- Os pesos do modelo são também salvos separadamente em formato `.h5`, o que pode ser útil para carregar apenas os pesos em outra arquitetura idêntica.

Estas práticas asseguram que o melhor modelo obtido durante o treino é guardado e reutilizável, tanto em produção como para avaliação futura.


In [ ]:
# One-hot encode test labels for evaluation
test_dataset_oh = test_dataset.map(one_hot_encode)

# Evaluate on test dataset
test_loss, test_acc = model.evaluate(test_dataset_oh)
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Get class names from your dataset
class_names = train_dataset.class_names
print("Classes:", class_names)

# Function to show predictions for a batch of images
plt.figure(figsize=(12, 12))
for images, labels in test_dataset.take(1):
    predictions = model.predict(images)
    pred_classes = np.argmax(predictions, axis=1)
    
    for i in range(24):
        plt.subplot(6, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        correct = labels[i] == pred_classes[i]
        color = "green" if correct else "red"
        
        plt.title(f"True: {class_names[labels[i]]}\nPred: {class_names[pred_classes[i]]}", 
                 color=color)
        plt.axis("off")
plt.tight_layout()
plt.show()